In [ ]:
from neo4j import GraphDatabase
import json
import time
import re
import uuid
from typing import Any, Dict, List, Union
from difflib import SequenceMatcher
from collections import defaultdict

# =====================================================================
# STEP 3: ENTITY RESOLUTION (FIXED VERSION)
# =====================================================================

class GraphStats:
    """Helper class to collect and display graph statistics"""
    
    @staticmethod
    def get_stats(driver) -> Dict[str, Any]:
        """Collect comprehensive graph statistics"""
        stats = {}
        
        with driver.session() as session:
            # Node counts
            result = session.run("""
                MATCH (n)
                RETURN labels(n)[0] as label, count(*) as count
                ORDER BY count DESC
            """)
            node_counts = {record['label']: record['count'] for record in result}
            stats['nodes'] = node_counts
            stats['total_nodes'] = sum(node_counts.values())
            
            # Relationship counts
            result = session.run("""
                MATCH ()-[r]->()
                RETURN type(r) as rel_type, count(*) as count
                ORDER BY count DESC
            """)
            rel_counts = {record['rel_type']: record['count'] for record in result}
            stats['relationships'] = rel_counts
            stats['total_relationships'] = sum(rel_counts.values())
            
            # Person details
            result = session.run("""
                MATCH (p:Person)
                WITH count(p) as total,
                     sum(CASE WHEN p.email IS NOT NULL THEN 1 ELSE 0 END) as with_email,
                     sum(CASE WHEN p.email IS NULL THEN 1 ELSE 0 END) as without_email
                RETURN total, with_email, without_email
            """)
            person_stats = result.single()
            if person_stats:
                stats['persons'] = {
                    'total': person_stats['total'],
                    'with_email': person_stats['with_email'],
                    'without_email': person_stats['without_email']
                }
            
            # Organization details
            result = session.run("MATCH (o:Organization) RETURN count(o) as count")
            stats['organizations'] = result.single()['count']
            
            # Email details
            result = session.run("MATCH (e:Email) RETURN count(e) as count")
            stats['emails'] = result.single()['count']

            # Location details
            result = session.run("MATCH (l:Location) RETURN count(l) as count")
            stats['locations'] = result.single()['count']
            
        return stats
    
    @staticmethod
    def print_stats(stats: Dict[str, Any], title: str = "Graph Statistics"):
        """Print statistics in a formatted way"""
        print("\n" + "=" * 70)
        print(f"{title:^70}")
        print("=" * 70)
        
        # Overall counts
        print(f"\n📊 OVERALL:")
        print(f"  Total Nodes:         {stats['total_nodes']:,}")
        print(f"  Total Relationships: {stats['total_relationships']:,}")
        
        # Node breakdown
        print(f"\n📦 NODES BY TYPE:")
        for label, count in sorted(stats['nodes'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {label:20s} {count:,}")
        
        # Relationship breakdown
        print(f"\n🔗 RELATIONSHIPS BY TYPE:")
        for rel_type, count in sorted(stats['relationships'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {rel_type:25s} {count:,}")
        
        # Person details
        if 'persons' in stats:
            print(f"\n👥 PERSON DETAILS:")
            print(f"  Total Persons:       {stats['persons']['total']:,}")
            print(f"  With Email:          {stats['persons']['with_email']:,} ({stats['persons']['with_email']/stats['persons']['total']*100:.1f}%)")
            print(f"  Without Email:       {stats['persons']['without_email']:,} ({stats['persons']['without_email']/stats['persons']['total']*100:.1f}%)")
        
        # Other details
        print(f"\n📧 OTHER:")
        print(f"  Organizations:       {stats.get('organizations', 0):,}")
        print(f"  Emails:              {stats.get('emails', 0):,}")
        
        print("=" * 70)
    
    @staticmethod
    def print_comparison(before: Dict[str, Any], after: Dict[str, Any]):
        """Print before/after comparison"""
        print("\n" + "=" * 70)
        print(f"{'BEFORE vs AFTER ENTITY RESOLUTION':^70}")
        print("=" * 70)
        
        # Calculate changes
        node_reduction = before['total_nodes'] - after['total_nodes']
        node_reduction_pct = (node_reduction / before['total_nodes'] * 100) if before['total_nodes'] > 0 else 0
        
        print(f"\n📊 OVERALL CHANGES:")
        print(f"  Nodes:         {before['total_nodes']:,} → {after['total_nodes']:,} ({node_reduction:+,} nodes, {node_reduction_pct:.1f}% reduction)")
        print(f"  Relationships: {before['total_relationships']:,} → {after['total_relationships']:,}")
        
        # Node type changes
        print(f"\n📦 NODE CHANGES BY TYPE:")
        all_labels = set(before['nodes'].keys()) | set(after['nodes'].keys())
        for label in sorted(all_labels):
            before_count = before['nodes'].get(label, 0)
            after_count = after['nodes'].get(label, 0)
            change = after_count - before_count
            if change != 0:
                print(f"  {label:20s} {before_count:,} → {after_count:,} ({change:+,})")
        
        # Person changes
        if 'persons' in before and 'persons' in after:
            print(f"\n👥 PERSON CHANGES:")
            before_persons = before['persons']['total']
            after_persons = after['persons']['total']
            merged = before_persons - after_persons
            print(f"  Total:       {before_persons:,} → {after_persons:,}")
            print(f"  Merged:      {merged:,} duplicate(s) removed")
            if merged > 0:
                print(f"  Reduction:   {merged/before_persons*100:.1f}% of persons were duplicates")
        
        # Organization changes
        before_orgs = before.get('organizations', 0)
        after_orgs = after.get('organizations', 0)
        if before_orgs != after_orgs:
            merged_orgs = before_orgs - after_orgs
            print(f"\n🏢 ORGANIZATION CHANGES:")
            print(f"  Total:       {before_orgs:,} → {after_orgs:,}")
            print(f"  Merged:      {merged_orgs:,} duplicate(s) removed")
            if merged_orgs > 0:
                print(f"  Reduction:   {merged_orgs/before_orgs*100:.1f}% of organizations were duplicates")
        
        print("=" * 70)


class EntityResolver:
    """Rule-based entity resolution for Neo4j knowledge graph - FIXED VERSION"""
    
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.stats_before = None
        self.stats_after = None
    
    def close(self):
        self.driver.close()

    @staticmethod
    def normalize_string(s: str) -> str:
        """Normalize string for comparison, removing common suffixes"""
        if not s:
            return ""
        s = s.lower().strip()
        s = re.sub(r'[^\w\s]', ' ', s)
        s = re.sub(r'\s+', ' ', s).strip()
        
        # Remove common company suffixes
        suffixes = [' inc', ' llc', ' lp', ' llp', ' corp', ' corporation', ' co', ' ltd', ' plc']
        for suffix in suffixes:
            if s.endswith(suffix):
                s = s[:-len(suffix)].strip()
        return s


        
    @staticmethod
    def similarity_score(s1: str, s2: str) -> float:
        """Calculate similarity between two strings (0-1)"""
        s1_norm = EntityResolver.normalize_string(s1)
        s2_norm = EntityResolver.normalize_string(s2)
        return SequenceMatcher(None, s1_norm, s2_norm).ratio()
    
    def find_duplicate_persons(self, threshold: float = 0.9) -> List[tuple]:
        """
        Find potential duplicate persons using blocking for efficiency
        FIXED: Uses blocking to reduce O(n²) to near-linear time
        """
        duplicates = []
        
        with self.driver.session() as session:
            result = session.run("""
                MATCH (p:Person)
                OPTIONAL MATCH (p)-[:AFFILIATED_WITH]->(o:Organization)
                RETURN p.key as key, p.name as name, p.email as email, 
                       collect(DISTINCT o.name) as orgs
            """)
            
            persons = list(result)
            
            # Block by email domain and name for efficiency
            email_blocks = defaultdict(list)
            name_blocks = defaultdict(list)
            
            for p in persons:
                # CRITICAL: Skip invalid keys
                if not p['key'] or p['key'] == 'None' or not isinstance(p['key'], str):
                    continue
                
                # Skip auto-generated unknown keys
                if p['key'].startswith('unknown:'):
                    continue
                    
                # Block by email domain
                if p.get('email') and '@' in p['email']:
                    domain = p['email'].split('@')[-1].lower()
                    if domain:
                        email_blocks[domain].append(p)
                
                # Block by first word of normalized name
                if p.get('name') and p['name'].lower() not in ['unknown', '[unknown]']:
                    normalized = self.normalize_string(p['name'])
                    if normalized:
                        first_word = normalized.split()[0] if normalized.split() else ''
                        if first_word and len(first_word) > 2:  # Skip very short names
                            name_blocks[first_word].append(p)
            
            # Compare within blocks only
            seen = set()
            
            # Process email blocks first (higher confidence)
            for block in email_blocks.values():
                for i, p1 in enumerate(block):
                    for p2 in block[i+1:]:
                        pair = tuple(sorted([p1['key'], p2['key']]))
                        if pair in seen:
                            continue
                        seen.add(pair)
                        
                        # Exact email match = definite duplicate
                        if p1.get('email') and p2.get('email'):
                            if p1['email'].lower() == p2['email'].lower():
                                duplicates.append((p1['key'], p2['key'], 1.0))
                                continue
                        
                        # Name similarity
                        if p1.get('name') and p2.get('name'):
                            name_sim = self.similarity_score(p1['name'], p2['name'])
                            if name_sim >= threshold:
                                duplicates.append((p1['key'], p2['key'], name_sim))
            
            # Process name blocks (lower confidence, higher threshold)
            for block in name_blocks.values():
                for i, p1 in enumerate(block):
                    for p2 in block[i+1:]:
                        pair = tuple(sorted([p1['key'], p2['key']]))
                        if pair in seen:
                            continue
                        seen.add(pair)
                        
                        # Higher threshold for name-only matches
                        name_sim = self.similarity_score(p1.get('name', ''), p2.get('name', ''))
                        if name_sim >= min(threshold + 0.05, 0.98):  # Stricter for name-only
                            duplicates.append((p1['key'], p2['key'], name_sim))
        
        return duplicates
    
    def merge_persons(self, keep_key: str, merge_key: str):
        """Merge two person nodes - FIXED: Added validation"""
        # CRITICAL: Validate keys before merging
        if not keep_key or not merge_key:
            print(f"  ✗ Skipping invalid merge: empty key")
            return
        
        if str(keep_key) == 'None' or str(merge_key) == 'None':
            print(f"  ✗ Skipping invalid merge: None key")
            return
            
        if keep_key == merge_key:
            print(f"  ✗ Skipping self-merge: {keep_key}")
            return
        
        # Skip merging unknown auto-generated keys
        if keep_key.startswith('unknown:') or merge_key.startswith('unknown:'):
            print(f"  ✗ Skipping auto-generated unknown key merge")
            return
        
        with self.driver.session() as session:
            session.execute_write(self._do_merge_persons, keep_key, merge_key)
    
    @staticmethod
    def _do_merge_persons(tx, keep_key: str, merge_key: str):
        """Transaction function to merge person nodes"""
        # Transfer SENT relationships
        tx.run("""
            MATCH (merge:Person {key: $merge_key})-[r:SENT]->(e:Email)
            MATCH (keep:Person {key: $keep_key})
            MERGE (keep)-[:SENT]->(e)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Transfer SENT_TO relationships
        tx.run("""
            MATCH (e:Email)-[r:SENT_TO]->(merge:Person {key: $merge_key})
            MATCH (keep:Person {key: $keep_key})
            MERGE (e)-[:SENT_TO]->(keep)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Transfer AFFILIATED_WITH relationships
        tx.run("""
            MATCH (merge:Person {key: $merge_key})-[r:AFFILIATED_WITH]->(o:Organization)
            MATCH (keep:Person {key: $keep_key})
            MERGE (keep)-[:AFFILIATED_WITH]->(o)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Merge properties (keep best available)
        tx.run("""
            MATCH (merge:Person {key: $merge_key})
            MATCH (keep:Person {key: $keep_key})
            SET keep.email = COALESCE(keep.email, merge.email),
                keep.name = CASE 
                    WHEN keep.name IN ['Unknown', '[unknown]', ''] THEN merge.name
                    ELSE keep.name
                END,
                keep.semantic_type = COALESCE(keep.semantic_type, merge.semantic_type)
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Delete merged node
        tx.run("MATCH (p:Person {key: $merge_key}) DETACH DELETE p", merge_key=merge_key)
    
    def find_duplicate_organizations(self, threshold: float = 0.85) -> List[tuple]:
        """Find duplicate organizations by normalized name similarity"""
        duplicates = []

        with self.driver.session() as session:
            result = session.run("MATCH (o:Organization) RETURN o.name as name")
            orgs = list(result)

            # Normalize names
            normalized_map = {}  # original -> normalized
            for org in orgs:
                if not org['name']:
                    continue
                normalized = self.normalize_string(org['name'])
                if normalized:
                    normalized_map[org['name']] = normalized

            seen = set()
            names = list(normalized_map.keys())
            for i, o1 in enumerate(names):
                for o2 in names[i+1:]:
                    n1 = normalized_map[o1]
                    n2 = normalized_map[o2]
                    pair = tuple(sorted([n1, n2]))
                    if pair in seen:
                        continue
                    seen.add(pair)

                    score = SequenceMatcher(None, n1, n2).ratio()
                    if score >= threshold:
                        # Always merge into the "alphabetically first" normalized name
                        keep = min(n1, n2)
                        merge = max(n1, n2)
                        duplicates.append((keep, merge, score))

        return duplicates
    
    def merge_organizations(self, keep_norm: str, merge_norm: str):
        """Merge organizations using normalized names"""
        # Convert normalized names back to actual node names
        with self.driver.session() as session:
            keep_node = session.run(
                "MATCH (o:Organization) WHERE toLower(o.name) CONTAINS $norm RETURN o.name as name",
                norm=keep_norm
            ).single()
            merge_node = session.run(
                "MATCH (o:Organization) WHERE toLower(o.name) CONTAINS $norm RETURN o.name as name",
                norm=merge_norm
            ).single()

        if keep_node and merge_node:
            self._do_merge_orgs(self.driver.session(), keep_node['name'], merge_node['name'])
    
    @staticmethod
    def _do_merge_orgs(tx, keep_name: str, merge_name: str):
        """Transaction function to merge organization nodes - FIXED"""
        # Transfer AFFILIATED_WITH relationships
        tx.run("""
            MATCH (p:Person)-[r:AFFILIATED_WITH]->(merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            MERGE (p)-[:AFFILIATED_WITH]->(keep)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        # Transfer SUBSIDIARY_OF relationships (incoming)
        tx.run("""
            MATCH (sub:Organization)-[r:SUBSIDIARY_OF]->(merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            MERGE (sub)-[:SUBSIDIARY_OF]->(keep)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        # Transfer SUBSIDIARY_OF relationship (outgoing) - FIXED
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            OPTIONAL MATCH (merge)-[r:SUBSIDIARY_OF]->(parent:Organization)
            WITH keep, parent, r
            WHERE parent IS NOT NULL
            MERGE (keep)-[:SUBSIDIARY_OF]->(parent)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        # Merge properties
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            SET keep.semantic_type = COALESCE(keep.semantic_type, merge.semantic_type),
                keep.role = COALESCE(keep.role, merge.role)
        """, keep_name=keep_name, merge_name=merge_name)
        
        # Delete merged node
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            DETACH DELETE merge
        """, merge_name=merge_name)
    
    def find_duplicate_locations(self,threshold:float=0.85)-> List[tuple]:
        """Find duplicate locations by normalized name similarity"""
        duplicates = []

        with self.driver.session() as session:
            result = session.run("MATCH (l:Location) RETURN l.name as name")
            locs = list(result)

            # Normalize names
            normalized_map = {}  # original -> normalized
            for loc in locs:
                if not loc['name']:
                    continue
                normalized = self.normalize_string(loc['name'])
                if normalized:
                    normalized_map[loc['name']] = normalized

            seen = set()
            names = list(normalized_map.keys())

            for i, n1_raw in enumerate(names):
                for n2_raw in names[i+1:]:
                    n1 = normalized_map[n1_raw]
                    n2 = normalized_map[n2_raw]

                    pair = tuple(sorted([n1, n2]))
                    if pair in seen:
                        continue
                    seen.add(pair)

                    score = SequenceMatcher(None, n1, n2).ratio()
                    if score >= threshold:
                        keep = min(n1, n2)
                        merge = max(n1, n2)
                        duplicates.append((keep, merge, score))

        return duplicates
    
    def merge_locations(self,keep_norm:str,merge_norm:str):
        """Merge location nodes using normalized names"""
        with self.driver.session() as session:
            keep_node = session.run(
                "MATCH (l:Location) WHERE toLower(l.name) CONTAINS $norm RETURN l.name as name",
                norm=keep_norm
            ).single()

            merge_node = session.run(
                "MATCH (l:Location) WHERE toLower(l.name) CONTAINS $norm RETURN l.name as name",
                norm=merge_norm
            ).single()

        if keep_node and merge_node:
            self._do_merge_locs(self.driver.session(), keep_node['name'], merge_node['name'])

    def _do_merge_locs(tx,keep_name:str,merge_name:str):
        """Transaction function to merge Location nodes"""
    
        # Transfer relationships FROM the duplicate TO the keeper
        tx.run("""
            MATCH (merge:Location {name: $merge_name})<-[r]-(n)
            MATCH (keep:Location {name: $keep_name})
            MERGE (n)-[new_r:LOCATED_IN]->(keep)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)

        tx.run("""
            MATCH (merge:Location {name: $merge_name})-[r]->(n)
            MATCH (keep:Location {name: $keep_name})
            MERGE (keep)-[new_r:LOCATED_IN]->(n)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)

        # Merge basic properties
        tx.run("""
            MATCH (merge:Location {name: $merge_name})
            MATCH (keep:Location {name: $keep_name})
            SET keep.semantic_type = COALESCE(keep.semantic_type, merge.semantic_type)
        """, keep_name=keep_name, merge_name=merge_name)
        
        # Delete the duplicate node
        tx.run("""
            MATCH (merge:Location {name: $merge_name})
            DETACH DELETE merge
        """, merge_name=merge_name)


    def preview_duplicates(self, person_threshold: float = 0.9, org_threshold: float = 0.85):
        """Preview what duplicates would be found without merging"""
        print("\n" + "=" * 70)
        print(f"{'DUPLICATE PREVIEW (No Merging)':^70}")
        print("=" * 70)
        
        # Get current stats
        stats = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(stats, "CURRENT GRAPH STATE")
        
        # Preview person duplicates
        print(f"\n👥 POTENTIAL PERSON DUPLICATES (threshold={person_threshold}):")
        person_dupes = self.find_duplicate_persons(person_threshold)
        
        if not person_dupes:
            print("  ✓ No person duplicates found!")
        else:
            print(f"  Found {len(person_dupes)} potential duplicate pair(s):\n")
            
            # Fetch details for better display
            with self.driver.session() as session:
                shown = 0
                for i, (key1, key2, score) in enumerate(person_dupes, 1):
                    if shown >= 10:  # Only show first 10
                        break
                    
                    # Check if both persons still exist
                    p1 = session.run(
                        "MATCH (p:Person {key: $key}) RETURN p.name as name, p.email as email",
                        key=key1
                    ).single()
                    p2 = session.run(
                        "MATCH (p:Person {key: $key}) RETURN p.name as name, p.email as email",
                        key=key2
                    ).single()
                    
                    # Skip if either person was already deleted
                    if not p1 or not p2:
                        continue
                    
                    shown += 1
                    print(f"  [{shown}] Similarity: {score:.3f}")
                    print(f"      Person A: {p1['name']} ({p1['email'] or 'no email'})")
                    print(f"      Person B: {p2['name']} ({p2['email'] or 'no email'})")
                    print()
            
            if len(person_dupes) > 10:
                print(f"  ... and {len(person_dupes) - 10} more duplicate pair(s)")
        
        # Preview organization duplicates
        print(f"\n🏢 POTENTIAL ORGANIZATION DUPLICATES (threshold={org_threshold}):")
        org_dupes = self.find_duplicate_organizations(org_threshold)
        
        if not org_dupes:
            print("  ✓ No organization duplicates found!")
        else:
            print(f"  Found {len(org_dupes)} potential duplicate pair(s):\n")
            
            with self.driver.session() as session:
                shown = 0
                for i, (name1, name2, score) in enumerate(org_dupes, 1):
                    if shown >= 10:
                        break
                    
                    # Check if both orgs still exist
                    o1 = session.run(
                        "MATCH (o:Organization {name: $name}) RETURN o.name as name",
                        name=name1
                    ).single()
                    o2 = session.run(
                        "MATCH (o:Organization {name: $name}) RETURN o.name as name",
                        name=name2
                    ).single()
                    
                    if not o1 or not o2:
                        continue
                    
                    shown += 1
                    print(f"  [{shown}] Similarity: {score:.3f}")
                    print(f"      Org A: {name1}")
                    print(f"      Org B: {name2}")
                    print()
            
            if len(org_dupes) > 10:
                print(f"  ... and {len(org_dupes) - 10} more duplicate pair(s)")
        
        # preview location duplicate
        print(f"\n📍 POTENTIAL LOCATION DUPLICATES (threshold={org_threshold}):")
        loc_dupes = self.find_duplicate_locations(org_threshold)

        if not loc_dupes:
            print("  ✓ No location duplicates found!")
        else:
            print(f"  Found {len(loc_dupes)} potential duplicate pair(s):\n")

            with self.driver.session() as session:
                shown = 0
                for i, (name1, name2, score) in enumerate(loc_dupes, 1):
                    if shown >= 10:
                        break

                    l1 = session.run(
                        "MATCH (l:Location {name: $name}) RETURN l.name as name",
                        name=name1
                    ).single()
                    l2 = session.run(
                        "MATCH (l:Location {name: $name}) RETURN l.name as name",
                        name=name2
                    ).single()

                    if not l1 or not l2:
                        continue

                    shown += 1
                    print(f"  [{shown}] Similarity: {score:.3f}")
                    print(f"      Loc A: {name1}")
                    print(f"      Loc B: {name2}")
                    print()

            if len(loc_dupes) > 10:
                print(f"  ... and {len(loc_dupes) - 10} more duplicate pair(s)")


        # Summary
        total_potential_merges = len(person_dupes) + len(org_dupes) + len(loc_dupes)
        print("\n" + "=" * 70)
        print(f"SUMMARY: {total_potential_merges} potential merge(s) identified")
        print(f"  • {len(person_dupes)} person merge(s)")
        print(f"  • {len(org_dupes)} organization merge(s)")
        print(f"  • {len(loc_dupes)} location merge(s)")

        if total_potential_merges > 0:
            estimated_reduction = len(person_dupes) + len(org_dupes) + len(loc_dupes)
            print(f"\nEstimated node reduction: {estimated_reduction} nodes")
        
        print("=" * 70)
    
    def resolve_all(self, person_threshold: float = 0.95, org_threshold: float = 0.90):
        """Run entity resolution on all entity types"""
        print("\n" + "=" * 70)
        print(f"{'ENTITY RESOLUTION STARTING':^70}")
        print("=" * 70)
        
        # Capture BEFORE stats
        print("\n[1/4] Capturing BEFORE statistics...")
        self.stats_before = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(self.stats_before, "BEFORE ENTITY RESOLUTION")
        
        start_time = time.time()
        
        # Persons
        print(f"\n[2/4] Resolving Persons (threshold={person_threshold})...")
        person_dupes = self.find_duplicate_persons(person_threshold)
        print(f"  Found {len(person_dupes)} potential person duplicates")
        
        person_merge_count = 0
        person_skip_count = 0
        for keep, merge, score in person_dupes:
            try:
                # Validate before attempting merge
                if not keep or not merge or keep == merge:
                    person_skip_count += 1
                    continue
                if str(keep) == 'None' or str(merge) == 'None':
                    person_skip_count += 1
                    continue
                
                print(f"    Merging {merge} -> {keep} (similarity: {score:.3f})")
                self.merge_persons(keep, merge)
                person_merge_count += 1
            except Exception as e:
                print(f"    ✗ Failed to merge {merge} -> {keep}: {e}")
                person_skip_count += 1
        
        print(f"  ✓ Successfully merged {person_merge_count} person node(s)")
        if person_skip_count > 0:
            print(f"  ⚠ Skipped {person_skip_count} invalid merge(s)")
        
        # Organizations
        print(f"\n[3/4] Resolving Organizations (threshold={org_threshold})...")
        org_dupes = self.find_duplicate_organizations(org_threshold)
        print(f"  Found {len(org_dupes)} potential organization duplicates")
        
        org_merge_count = 0
        org_skip_count = 0
        for keep, merge, score in org_dupes:
            try:
                if not keep or not merge or keep == merge:
                    org_skip_count += 1
                    continue
                
                print(f"    Merging '{merge}' -> '{keep}' (similarity: {score:.3f})")
                self.merge_organizations(keep, merge)
                org_merge_count += 1
            except Exception as e:
                print(f"    ✗ Failed to merge '{merge}' -> '{keep}': {e}")
                org_skip_count += 1
        
        print(f"  ✓ Successfully merged {org_merge_count} organization node(s)")
        if org_skip_count > 0:
            print(f"  ⚠ Skipped {org_skip_count} invalid merge(s)")
        
        # Locations
        print(f"\n[4/5] Resolving Locations (threshold={org_threshold})...")
        loc_dupes = self.find_duplicate_locations(org_threshold)
        print(f"  Found {len(loc_dupes)} potential location duplicates")

        loc_merge_count = 0
        loc_skip_count = 0
        for keep, merge, score in loc_dupes:
            try:
                if not keep or not merge or keep == merge:
                    loc_skip_count += 1
                    continue

                print(f"    Merging '{merge}' -> '{keep}' (similarity: {score:.3f})")
                self.merge_locations(keep, merge)
                loc_merge_count += 1
            except Exception as e:
                print(f"    ✗ Failed to merge '{merge}' -> '{keep}': {e}")
                loc_skip_count += 1

        print(f"  ✓ Successfully merged {loc_merge_count} location node(s)")
        if loc_skip_count > 0:
            print(f"  ⚠ Skipped {loc_skip_count} invalid merge(s)")


        # Capture AFTER stats
        print(f"\n[4/4] Capturing AFTER statistics...")
        self.stats_after = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(self.stats_after, "AFTER ENTITY RESOLUTION")
        
        # Print comparison
        GraphStats.print_comparison(self.stats_before, self.stats_after)
        
        elapsed = time.time() - start_time
        print(f"\n⏱️  Entity Resolution completed in {elapsed:.2f} seconds")
        print("=" * 70)


In [ ]:
# =====================================================================
# MAIN PIPELINE
# =====================================================================

def main():
    # Configuration
    JSONL_PATH = "C:\\Users\\aditi\\source\\repos\\ETL\\qwen_output_v1.jsonl"
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PW = "neo4j@1234"
    
    print("=" * 70)
    print("KNOWLEDGE GRAPH IMPORT & ENTITY RESOLUTION PIPELINE (FIXED)")
    print("=" * 70)
    
    from ETL.hayleyQueries_v2 import import_jsonl_to_neo4j
    import_jsonl_to_neo4j(JSONL_PATH, NEO4J_URI,NEO4J_USER,NEO4J_PW,50)
    # STEP 1: Import all data (creates duplicates)
    print("\n[STEP 1/3] Importing JSONL data...")
    '''import_jsonl_to_neo4j(
        jsonl_path=JSONL_PATH,
        uri=NEO4J_URI,
        user=NEO4J_USER,
        password=NEO4J_PW,
        log_every=50
    )'''
    
    # STEP 2: Preview duplicates (optional - shows what will be merged)
    print("\n[STEP 2/3] Previewing potential duplicates...")
    resolver = EntityResolver(NEO4J_URI, NEO4J_USER, NEO4J_PW)
    try:
        resolver.preview_duplicates(
            person_threshold=0.95,  # Conservative threshold
            org_threshold=0.90
        )
        
        # Ask user if they want to proceed (optional)
        print("\n" + "=" * 70)
        response = input("Proceed with entity resolution? (yes/no): ").lower()
        if response not in ['yes', 'y']:
            print("Entity resolution cancelled.")
            return
        
        # STEP 3: Entity resolution (merges duplicates)
        print("\n[STEP 3/3] Running entity resolution...")
        resolver.resolve_all(
            person_threshold=0.95,  # High threshold = only merge very similar
            org_threshold=0.90
        )
        
    finally:
        resolver.close()
    
    print("\n" + "=" * 70)
    print("PIPELINE COMPLETE!")
    print("=" * 70)


if __name__ == "__main__":
    main()

KNOWLEDGE GRAPH IMPORT & ENTITY RESOLUTION PIPELINE (FIXED)
[INFO] Processing line 50... (success=49, failed=0, skipped=0)
	 took 28.716962337493896 seconds
[INFO] Processing line 100... (success=99, failed=0, skipped=0)
	 took 55.50732350349426 seconds
[INFO] Processing line 150... (success=149, failed=0, skipped=0)
	 took 77.38678216934204 seconds
[INFO] Processing line 200... (success=199, failed=0, skipped=0)
	 took 107.65881538391113 seconds
[INFO] Processing line 250... (success=249, failed=0, skipped=0)
	 took 132.98851919174194 seconds
[INFO] Processing line 300... (success=299, failed=0, skipped=0)
	 took 161.80616307258606 seconds
[INFO] Processing line 350... (success=349, failed=0, skipped=0)
	 took 185.7807993888855 seconds
[INFO] Processing line 400... (success=399, failed=0, skipped=0)
	 took 208.31332802772522 seconds
[INFO] Processing line 450... (success=449, failed=0, skipped=0)
	 took 233.18606424331665 seconds
[INFO] Processing line 500... (success=499, failed=0, s

c:\Users\aditi\source\repos\.venv\lib\site-packages\neo4j\_sync\work\result.py:625: UserWarning: Expected a result with a single record, but found multiple.
  warn(


    Merging 'norwalk2' -> 'norwalk' (similarity: 0.933)
    Merging 'national association of boards of pharmacy nabp' -> 'national association of boards of pharmacy' (similarity: 0.944)
    Merging 'walmart' -> 'wal mart' (similarity: 0.933)
    Merging 'walgreens' -> 'walgreen' (similarity: 0.941)
    Merging 'healthcare distribution management association hdma' -> 'healthcare distribution management association' (similarity: 0.948)
    Merging 'qualitestrx' -> 'qualitest' (similarity: 0.900)
    Merging 'teva pharmaceuticals' -> 'teva pharmaceutical' (similarity: 0.974)
    Merging 'teva pharmaceuticals usa' -> 'teva pharmaceuticals' (similarity: 0.909)
    Merging 'us department of justice' -> 'u s department of justice' (similarity: 0.980)
    Merging 'covidien mallinckrodt pharmaceuticals' -> 'covidien mallinckrodt pharmaceutical products' (similarity: 0.902)
    Merging 'mallinckrodt pharmaceutical generics' -> 'mallinckrodt dosage pharmaceuticals generics' (similarity: 0.900)
  